In [8]:
import pandas as pd
import numpy as np
import ast
import re
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

In [9]:
df = pd.read_csv('gb_game_reviews.csv')
df.head()

,date_added,deck,description,reviewer,score,site_detail_url,wikiObject,characters,concepts,deck.1,description.1,developers,franchises,genres,name,platforms,publishers,similar_games,themes
0,2008-07-14 21:49:21,Review Monster,"<p>Hey, I bet this gives no tangible notice th...",snide,5.0,https://www.giantbomb.com/pac-man/3030-7624/us...,Famicom Mini: Pac-Man,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,One of the most popular arcade games of all ti...,"<h2>Overview</h2><figure data-align=""right"" da...",[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,Pac-Man,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,NaN
1,2008-07-20 01:09:02,My Favorite Combat RPG of All Time,<div>Rambling on about one of my favorite game...,snide,5.0,https://www.giantbomb.com/icewind-dale-ii/3030...,Icewind Dale II (Collector's Edition),NaN,[{'api_detail_url': 'https://www.giantbomb.com...,Icewind Dale II was the last game to be built ...,<h2>Summary</h2><p>Icewind Dale II is an <a da...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,Icewind Dale II,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...
2,2008-07-21 00:34:06,Simple But Fun,<p>There’s something to be said for single-min...,joseppie,4.0,https://www.giantbomb.com/gauntlet-legends/303...,Gauntlet Legends,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,Gauntlet Legends is the first 3D installment i...,<h2>Story</h2>A mage named Garm summoned the d...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,Gauntlet Legends,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...
3,2008-07-21 00:36:10,The Best Around (Still),<p>The last and best of the MVP Baseball Serie...,joseppie,5.0,https://www.giantbomb.com/mvp-baseball-2005/30...,MVP Baseball 2005,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,MVP Baseball 2005 is a baseball game from EA S...,<h2>New Features<br /></h2><h3>Owner’s Mode</h...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,MVP Baseball 2005,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,NaN
4,2008-07-21 00:44:42,Thrills and Hilarity Ensue...,<p>Need for Speed: Most Wanted acts as an inje...,joseppie,4.0,https://www.giantbomb.com/need-for-speed-most-...,Need for Speed: Most Wanted,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,"Cruise the streets of Rockport City, keep away...","<h2>Overview</h2><figure data-align=""right"" da...",[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,Need for Speed: Most Wanted,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26914 entries, 0 to 26913
Data columns (total 19 columns):
date_added         26914 non-null object
deck               26914 non-null object
description        26838 non-null object
reviewer           26914 non-null object
score              26914 non-null float64
site_detail_url    26914 non-null object
wikiObject         26914 non-null object
characters         23023 non-null object
concepts           26821 non-null object
deck.1             26880 non-null object
description.1      26535 non-null object
developers         26784 non-null object
franchises         22287 non-null object
genres             26874 non-null object
name               26914 non-null object
platforms          26912 non-null object
publishers         26703 non-null object
similar_games      25497 non-null object
themes             24126 non-null object
dtypes: float64(1), object(18)
memory usage: 3.9+ MB


In [11]:
df = df.drop(['wikiObject', 'site_detail_url'], axis=1) # remove irrelevant features
df.head()

,date_added,deck,description,reviewer,score,characters,concepts,deck.1,description.1,developers,franchises,genres,name,platforms,publishers,similar_games,themes
0,2008-07-14 21:49:21,Review Monster,"<p>Hey, I bet this gives no tangible notice th...",snide,5.0,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,One of the most popular arcade games of all ti...,"<h2>Overview</h2><figure data-align=""right"" da...",[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,Pac-Man,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,NaN
1,2008-07-20 01:09:02,My Favorite Combat RPG of All Time,<div>Rambling on about one of my favorite game...,snide,5.0,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,Icewind Dale II was the last game to be built ...,<h2>Summary</h2><p>Icewind Dale II is an <a da...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,Icewind Dale II,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...
2,2008-07-21 00:34:06,Simple But Fun,<p>There’s something to be said for single-min...,joseppie,4.0,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,Gauntlet Legends is the first 3D installment i...,<h2>Story</h2>A mage named Garm summoned the d...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,Gauntlet Legends,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...
3,2008-07-21 00:36:10,The Best Around (Still),<p>The last and best of the MVP Baseball Serie...,joseppie,5.0,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,MVP Baseball 2005 is a baseball game from EA S...,<h2>New Features<br /></h2><h3>Owner’s Mode</h...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,MVP Baseball 2005,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,NaN
4,2008-07-21 00:44:42,Thrills and Hilarity Ensue...,<p>Need for Speed: Most Wanted acts as an inje...,joseppie,4.0,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,"Cruise the streets of Rockport City, keep away...","<h2>Overview</h2><figure data-align=""right"" da...",[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,Need for Speed: Most Wanted,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...


In [12]:
# rename features to prevent duplicate column names
df = df.rename(columns={"deck": "rev_deck", "description": "rev_body", "deck.1" : "game_deck", "description.1" : "game_body"})
df.head()

,date_added,rev_deck,rev_body,reviewer,score,characters,concepts,game_deck,game_body,developers,franchises,genres,name,platforms,publishers,similar_games,themes
0,2008-07-14 21:49:21,Review Monster,"<p>Hey, I bet this gives no tangible notice th...",snide,5.0,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,One of the most popular arcade games of all ti...,"<h2>Overview</h2><figure data-align=""right"" da...",[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,Pac-Man,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,NaN
1,2008-07-20 01:09:02,My Favorite Combat RPG of All Time,<div>Rambling on about one of my favorite game...,snide,5.0,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,Icewind Dale II was the last game to be built ...,<h2>Summary</h2><p>Icewind Dale II is an <a da...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,Icewind Dale II,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...
2,2008-07-21 00:34:06,Simple But Fun,<p>There’s something to be said for single-min...,joseppie,4.0,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,Gauntlet Legends is the first 3D installment i...,<h2>Story</h2>A mage named Garm summoned the d...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,Gauntlet Legends,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...
3,2008-07-21 00:36:10,The Best Around (Still),<p>The last and best of the MVP Baseball Serie...,joseppie,5.0,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,MVP Baseball 2005 is a baseball game from EA S...,<h2>New Features<br /></h2><h3>Owner’s Mode</h...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,MVP Baseball 2005,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,NaN
4,2008-07-21 00:44:42,Thrills and Hilarity Ensue...,<p>Need for Speed: Most Wanted acts as an inje...,joseppie,4.0,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,"Cruise the streets of Rockport City, keep away...","<h2>Overview</h2><figure data-align=""right"" da...",[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,Need for Speed: Most Wanted,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...


In [13]:
"""
Extract just the name of the game feature and return it as a string

Input: feature value, a list of items that including the name/names of the feature and links (which are disregarded)

Return name/names as string
"""
def extract_name(value):
    if type(value) == float: # if feature value is null
        return(np.nan)
    else:
        value = ast.literal_eval(value) # convert string list to list
        string = ''
        for x in range(len(value)):
            name = value[x]['name'] # extract name/names
            if len(name.split()) > 1:
                name = '-'.join([word for word in name.split()]) # attach - to names with more than one word
            string += (name + ' ')
        return(string)

In [14]:
# extract name/names for all game features
df['genres'] = df['genres'].apply(lambda x: extract_name(x))
df['developers'] = df['developers'].apply(lambda x: extract_name(x))
df['franchises'] = df['franchises'].apply(lambda x: extract_name(x))
df['platforms'] = df['platforms'].apply(lambda x: extract_name(x))
df['publishers'] = df['publishers'].apply(lambda x: extract_name(x))
df['characters'] = df['characters'].apply(lambda x: extract_name(x))
df['concepts'] = df['concepts'].apply(lambda x: extract_name(x))
df['similar_games'] = df['similar_games'].apply(lambda x: extract_name(x))
df['themes'] = df['themes'].apply(lambda x: extract_name(x))

In [15]:
df = df[df.similar_games.notnull()] # remove records with null similar games list
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25497 entries, 0 to 26913
Data columns (total 17 columns):
date_added       25497 non-null object
rev_deck         25497 non-null object
rev_body         25423 non-null object
reviewer         25497 non-null object
score            25497 non-null float64
characters       22008 non-null object
concepts         25459 non-null object
game_deck        25475 non-null object
game_body        25247 non-null object
developers       25411 non-null object
franchises       21189 non-null object
genres           25480 non-null object
name             25497 non-null object
platforms        25497 non-null object
publishers       25345 non-null object
similar_games    25497 non-null object
themes           23065 non-null object
dtypes: float64(1), object(16)
memory usage: 3.5+ MB


In [16]:
df.name = df.name.apply(lambda x: '-'.join(x.split(' '))) # attach - to game names with more than one word

In [17]:
"""
Removes html tags from text

Input: raw_html string, or text

Returns string without html tags
"""
def cleanhtml(raw_html):
    if type(raw_html) == float:
        return np.nan
    else:
        cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
        cleantext = re.sub(cleanr, ' ', raw_html)
        return cleantext

In [18]:
df['rev_body'] = df['rev_body'].apply(lambda x: cleanhtml(x)) # remove html tags from game reviews and game description
df['game_body'] = df['game_body'].apply(lambda x: cleanhtml(x))

In [19]:
df.head()

,date_added,rev_deck,rev_body,reviewer,score,characters,concepts,game_deck,game_body,developers,franchises,genres,name,platforms,publishers,similar_games,themes
0,2008-07-14 21:49:21,Review Monster,"Hey, I bet this gives no tangible notice that...",snide,5.0,Blinky Pinky Pac-Man Inky Clyde,Cutscene Achievements Leaderboard Original-Sou...,One of the most popular arcade games of all ti...,Overview An important icon of arcade gam...,Namco WiZ,Pac-Man ARCADE-GAME-SERIES,Strategy Puzzle,Pac-Man,Game-Boy Game-Boy-Advance Game-Gear Apple-II C...,Bandai-Namco-Entertainment Thunder-Mountain Bu...,Amidar Wizard-of-Wor Crystal-Castles Lock-'n'-...,NaN
1,2008-07-20 01:09:02,My Favorite Combat RPG of All Time,Rambling on about one of my favorite games. N...,snide,5.0,NaN,Experience-Points Micromanagement Dialogue-Tre...,Icewind Dale II was the last game to be built ...,Summary Icewind Dale II is an isometric pa...,Black-Isle-Studios,Icewind-Dale Dungeons-&-Dragons Forgotten-Realms,Role-Playing,Icewind-Dale-II,PC,Interplay-Entertainment-Corp.,Planescape:-Torment Baldur's-Gate-II:-Shadows-...,Fantasy
2,2008-07-21 00:34:06,Simple But Fun,There’s something to be said for single-minde...,joseppie,4.0,Death Merlin-the-Wizard Sumner Garm Skorne Wiz...,Teleportation Cloak Experience-Points Boss-Fig...,Gauntlet Legends is the first 3D installment i...,Story A mage named Garm summoned the demon Sk...,Midway-Games,Gauntlet,Role-Playing Brawler,Gauntlet-Legends,PlayStation Dreamcast Nintendo-64 Arcade,Midway-Games,Phantasy-Star-Online-Episode-I-&-II-Plus Phant...,Fantasy
3,2008-07-21 00:36:10,The Best Around (Still),"The last and best of the MVP Baseball Series,...",joseppie,5.0,Roger-Clemens Derrek-Lee Jonathan-Papelbon Ich...,Cover-Athlete Instant-Replay Minigames Artific...,MVP Baseball 2005 is a baseball game from EA S...,New Features Owner’s Mode The existing fra...,EA-Canada,MVP-Baseball MLB,Sports Baseball,MVP-Baseball-2005,PlayStation-Portable PlayStation-2 GameCube Xb...,Electronic-Arts,MLB-08:-The-Show MLB-Power-Pros-2008 Major-Lea...,NaN
4,2008-07-21 00:44:42,Thrills and Hilarity Ensue...,Need for Speed: Most Wanted acts as an inject...,joseppie,4.0,Razor-Callahan Sergeant-Cross Mia-Townsend Lou...,Speed-Blur Cutscene Achievements Leaderboard R...,"Cruise the streets of Rockport City, keep away...",Overview Need for Speed: Most Wanted wa...,EA-Canada EA-Black-Box Ideaworks-Game-Studio,Need-for-Speed,Driving/Racing Simulation,Need-for-Speed:-Most-Wanted,Game-Boy-Advance Mac PlayStation-2 Xbox-360 Ga...,Electronic-Arts Pocketeers,Forza-Motorsport-2 Test-Drive-Unlimited Burnou...,Crime Motorsports


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25497 entries, 0 to 26913
Data columns (total 17 columns):
date_added       25497 non-null object
rev_deck         25497 non-null object
rev_body         25423 non-null object
reviewer         25497 non-null object
score            25497 non-null float64
characters       22008 non-null object
concepts         25459 non-null object
game_deck        25475 non-null object
game_body        25247 non-null object
developers       25411 non-null object
franchises       21189 non-null object
genres           25480 non-null object
name             25497 non-null object
platforms        25497 non-null object
publishers       25345 non-null object
similar_games    25497 non-null object
themes           23065 non-null object
dtypes: float64(1), object(16)
memory usage: 3.5+ MB


In [21]:
"""
Create list of feature names for each game feature

Input: string of feature names, each name separated by a space

Returns list of names
"""
def make_list(value):
    if type(value) == float:
        return(np.nan)
    else:
        value = value.split()
        return value

In [22]:
# convert each features name string to list of names
df['genres'] = df['genres'].apply(lambda x: make_list(x))
df['developers'] = df['developers'].apply(lambda x: make_list(x))
df['franchises'] = df['franchises'].apply(lambda x: make_list(x))
df['platforms'] = df['platforms'].apply(lambda x: make_list(x))
df['publishers'] = df['publishers'].apply(lambda x: make_list(x))
df['characters'] = df['characters'].apply(lambda x: make_list(x))
df['concepts'] = df['concepts'].apply(lambda x: make_list(x))
df['similar_games'] = df['similar_games'].apply(lambda x: make_list(x))
df['themes'] = df['themes'].apply(lambda x: make_list(x))

In [23]:
df.head()

,date_added,rev_deck,rev_body,reviewer,score,characters,concepts,game_deck,game_body,developers,franchises,genres,name,platforms,publishers,similar_games,themes
0,2008-07-14 21:49:21,Review Monster,"Hey, I bet this gives no tangible notice that...",snide,5.0,"[Blinky, Pinky, Pac-Man, Inky, Clyde]","[Cutscene, Achievements, Leaderboard, Original...",One of the most popular arcade games of all ti...,Overview An important icon of arcade gam...,"[Namco, WiZ]","[Pac-Man, ARCADE-GAME-SERIES]","[Strategy, Puzzle]",Pac-Man,"[Game-Boy, Game-Boy-Advance, Game-Gear, Apple-...","[Bandai-Namco-Entertainment, Thunder-Mountain,...","[Amidar, Wizard-of-Wor, Crystal-Castles, Lock-...",NaN
1,2008-07-20 01:09:02,My Favorite Combat RPG of All Time,Rambling on about one of my favorite games. N...,snide,5.0,NaN,"[Experience-Points, Micromanagement, Dialogue-...",Icewind Dale II was the last game to be built ...,Summary Icewind Dale II is an isometric pa...,[Black-Isle-Studios],"[Icewind-Dale, Dungeons-&-Dragons, Forgotten-R...",[Role-Playing],Icewind-Dale-II,[PC],[Interplay-Entertainment-Corp.],"[Planescape:-Torment, Baldur's-Gate-II:-Shadow...",[Fantasy]
2,2008-07-21 00:34:06,Simple But Fun,There’s something to be said for single-minde...,joseppie,4.0,"[Death, Merlin-the-Wizard, Sumner, Garm, Skorn...","[Teleportation, Cloak, Experience-Points, Boss...",Gauntlet Legends is the first 3D installment i...,Story A mage named Garm summoned the demon Sk...,[Midway-Games],[Gauntlet],"[Role-Playing, Brawler]",Gauntlet-Legends,"[PlayStation, Dreamcast, Nintendo-64, Arcade]",[Midway-Games],"[Phantasy-Star-Online-Episode-I-&-II-Plus, Pha...",[Fantasy]
3,2008-07-21 00:36:10,The Best Around (Still),"The last and best of the MVP Baseball Series,...",joseppie,5.0,"[Roger-Clemens, Derrek-Lee, Jonathan-Papelbon,...","[Cover-Athlete, Instant-Replay, Minigames, Art...",MVP Baseball 2005 is a baseball game from EA S...,New Features Owner’s Mode The existing fra...,[EA-Canada],"[MVP-Baseball, MLB]","[Sports, Baseball]",MVP-Baseball-2005,"[PlayStation-Portable, PlayStation-2, GameCube...",[Electronic-Arts],"[MLB-08:-The-Show, MLB-Power-Pros-2008, Major-...",NaN
4,2008-07-21 00:44:42,Thrills and Hilarity Ensue...,Need for Speed: Most Wanted acts as an inject...,joseppie,4.0,"[Razor-Callahan, Sergeant-Cross, Mia-Townsend,...","[Speed-Blur, Cutscene, Achievements, Leaderboa...","Cruise the streets of Rockport City, keep away...",Overview Need for Speed: Most Wanted wa...,"[EA-Canada, EA-Black-Box, Ideaworks-Game-Studio]",[Need-for-Speed],"[Driving/Racing, Simulation]",Need-for-Speed:-Most-Wanted,"[Game-Boy-Advance, Mac, PlayStation-2, Xbox-36...","[Electronic-Arts, Pocketeers]","[Forza-Motorsport-2, Test-Drive-Unlimited, Bur...","[Crime, Motorsports]"


In [45]:
unique = df.name.unique()
"""
Removes games from the similar games if the game has not been reviewed (not part of 4243 unique games)



"""
def remove_game(sim_list):
    for game in sim_list:
        if game not in unique:
            sim_list.remove(game)
        else:
            continue
    return sim_list

In [46]:
df['similar_games'] = df.similar_games.apply(lambda x: remove_game(x)) 

In [47]:
df = df[df.similar_games.apply(lambda x: len(x) != 0)] # if similar games contains no games, remove record
len(df)

24025

In [51]:
df = df[df.score != 0.5] # remove 0.5 ratings to keep integer number ratings 

In [52]:
# convert string to lowercase lettering
df['rev_deck'] = df['rev_deck'].str.lower()
df['rev_body'] = df['rev_body'].str.lower()
df['game_deck'] = df['game_deck'].str.lower()
df['game_body'] = df['game_body'].str.lower()

df.head()

,date_added,rev_deck,rev_body,reviewer,score,characters,concepts,game_deck,game_body,developers,franchises,genres,name,platforms,publishers,similar_games,themes
0,2008-07-14 21:49:21,review monster,"hey, i bet this gives no tangible notice that...",snide,5.0,"[Blinky, Pinky, Pac-Man, Inky, Clyde]","[Cutscene, Achievements, Leaderboard, Original...",one of the most popular arcade games of all ti...,overview an important icon of arcade gam...,"[Namco, WiZ]","[Pac-Man, ARCADE-GAME-SERIES]","[Strategy, Puzzle]",Pac-Man,"[Game-Boy, Game-Boy-Advance, Game-Gear, Apple-...","[Bandai-Namco-Entertainment, Thunder-Mountain,...","[Lock-'n'-Chase, Lady-Bug, Katamari-Damacy, Sp...",NaN
1,2008-07-20 01:09:02,my favorite combat rpg of all time,rambling on about one of my favorite games. n...,snide,5.0,NaN,"[Experience-Points, Micromanagement, Dialogue-...",icewind dale ii was the last game to be built ...,summary icewind dale ii is an isometric pa...,[Black-Isle-Studios],"[Icewind-Dale, Dungeons-&-Dragons, Forgotten-R...",[Role-Playing],Icewind-Dale-II,[PC],[Interplay-Entertainment-Corp.],"[Planescape:-Torment, Baldur's-Gate-II:-Shadow...",[Fantasy]
2,2008-07-21 00:34:06,simple but fun,there’s something to be said for single-minde...,joseppie,4.0,"[Death, Merlin-the-Wizard, Sumner, Garm, Skorn...","[Teleportation, Cloak, Experience-Points, Boss...",gauntlet legends is the first 3d installment i...,story a mage named garm summoned the demon sk...,[Midway-Games],[Gauntlet],"[Role-Playing, Brawler]",Gauntlet-Legends,"[PlayStation, Dreamcast, Nintendo-64, Arcade]",[Midway-Games],"[Champions-of-Norrath, Ghostbusters:-Sanctum-o...",[Fantasy]
3,2008-07-21 00:36:10,the best around (still),"the last and best of the mvp baseball series,...",joseppie,5.0,"[Roger-Clemens, Derrek-Lee, Jonathan-Papelbon,...","[Cover-Athlete, Instant-Replay, Minigames, Art...",mvp baseball 2005 is a baseball game from ea s...,new features owner’s mode the existing fra...,[EA-Canada],"[MVP-Baseball, MLB]","[Sports, Baseball]",MVP-Baseball-2005,"[PlayStation-Portable, PlayStation-2, GameCube...",[Electronic-Arts],"[MLB-08:-The-Show, Major-League-Baseball-2K8, ...",NaN
4,2008-07-21 00:44:42,thrills and hilarity ensue...,need for speed: most wanted acts as an inject...,joseppie,4.0,"[Razor-Callahan, Sergeant-Cross, Mia-Townsend,...","[Speed-Blur, Cutscene, Achievements, Leaderboa...","cruise the streets of rockport city, keep away...",overview need for speed: most wanted wa...,"[EA-Canada, EA-Black-Box, Ideaworks-Game-Studio]",[Need-for-Speed],"[Driving/Racing, Simulation]",Need-for-Speed:-Most-Wanted,"[Game-Boy-Advance, Mac, PlayStation-2, Xbox-36...","[Electronic-Arts, Pocketeers]","[Forza-Motorsport-2, Test-Drive-Unlimited, Bur...","[Crime, Motorsports]"


In [53]:
df = df.replace(np.nan, '', regex=True) # replace NaN with empty space

In [54]:
# convert text string to list of strings
tokenizer = RegexpTokenizer(r'\w+')

df['game_body'] = df['game_body'].apply(lambda x: tokenizer.tokenize(x))
df['game_deck'] = df['game_deck'].apply(lambda x: tokenizer.tokenize(x))
df['rev_body'] = df['rev_body'].apply(lambda x: tokenizer.tokenize(x))
df['rev_deck'] = df['rev_deck'].apply(lambda x: tokenizer.tokenize(x))

df.head(10)

,date_added,rev_deck,rev_body,reviewer,score,characters,concepts,game_deck,game_body,developers,franchises,genres,name,platforms,publishers,similar_games,themes
0,2008-07-14 21:49:21,"[review, monster]","[hey, i, bet, this, gives, no, tangible, notic...",snide,5.0,"[Blinky, Pinky, Pac-Man, Inky, Clyde]","[Cutscene, Achievements, Leaderboard, Original...","[one, of, the, most, popular, arcade, games, o...","[overview, an, important, icon, of, arcade, ga...","[Namco, WiZ]","[Pac-Man, ARCADE-GAME-SERIES]","[Strategy, Puzzle]",Pac-Man,"[Game-Boy, Game-Boy-Advance, Game-Gear, Apple-...","[Bandai-Namco-Entertainment, Thunder-Mountain,...","[Lock-'n'-Chase, Lady-Bug, Katamari-Damacy, Sp...",
1,2008-07-20 01:09:02,"[my, favorite, combat, rpg, of, all, time]","[rambling, on, about, one, of, my, favorite, g...",snide,5.0,,"[Experience-Points, Micromanagement, Dialogue-...","[icewind, dale, ii, was, the, last, game, to, ...","[summary, icewind, dale, ii, is, an, isometric...",[Black-Isle-Studios],"[Icewind-Dale, Dungeons-&-Dragons, Forgotten-R...",[Role-Playing],Icewind-Dale-II,[PC],[Interplay-Entertainment-Corp.],"[Planescape:-Torment, Baldur's-Gate-II:-Shadow...",[Fantasy]
2,2008-07-21 00:34:06,"[simple, but, fun]","[there, s, something, to, be, said, for, singl...",joseppie,4.0,"[Death, Merlin-the-Wizard, Sumner, Garm, Skorn...","[Teleportation, Cloak, Experience-Points, Boss...","[gauntlet, legends, is, the, first, 3d, instal...","[story, a, mage, named, garm, summoned, the, d...",[Midway-Games],[Gauntlet],"[Role-Playing, Brawler]",Gauntlet-Legends,"[PlayStation, Dreamcast, Nintendo-64, Arcade]",[Midway-Games],"[Champions-of-Norrath, Ghostbusters:-Sanctum-o...",[Fantasy]
3,2008-07-21 00:36:10,"[the, best, around, still]","[the, last, and, best, of, the, mvp, baseball,...",joseppie,5.0,"[Roger-Clemens, Derrek-Lee, Jonathan-Papelbon,...","[Cover-Athlete, Instant-Replay, Minigames, Art...","[mvp, baseball, 2005, is, a, baseball, game, f...","[new, features, owner, s, mode, the, existing,...",[EA-Canada],"[MVP-Baseball, MLB]","[Sports, Baseball]",MVP-Baseball-2005,"[PlayStation-Portable, PlayStation-2, GameCube...",[Electronic-Arts],"[MLB-08:-The-Show, Major-League-Baseball-2K8, ...",
4,2008-07-21 00:44:42,"[thrills, and, hilarity, ensue]","[need, for, speed, most, wanted, acts, as, an,...",joseppie,4.0,"[Razor-Callahan, Sergeant-Cross, Mia-Townsend,...","[Speed-Blur, Cutscene, Achievements, Leaderboa...","[cruise, the, streets, of, rockport, city, kee...","[overview, need, for, speed, most, wanted, was...","[EA-Canada, EA-Black-Box, Ideaworks-Game-Studio]",[Need-for-Speed],"[Driving/Racing, Simulation]",Need-for-Speed:-Most-Wanted,"[Game-Boy-Advance, Mac, PlayStation-2, Xbox-36...","[Electronic-Arts, Pocketeers]","[Forza-Motorsport-2, Test-Drive-Unlimited, Bur...","[Crime, Motorsports]"
5,2008-07-21 00:49:38,"[to, call, this, game, amazing, would, be, a, ...","[grand, theft, auto, in, my, eyes, is, one, of...",toriko,5.0,"[Roman-Bellic, Playboy-X, Niko-Bellic, Little-...","[Multiple-Endings, Save-Point, Day/Night-Cycle...","[take, on, the, role, of, niko, bellic, a, ser...","[overview, welcome, to, liberty, city, the, gr...",[Rockstar-North],[Grand-Theft-Auto],"[Action, Shooter]",Grand-Theft-Auto-IV,"[Xbox-360, PlayStation-3, PC]",[Rockstar-Games],"[Mafia-III, The-Godfather:-The-Game]",[Crime]
6,2008-07-21 00:51:31,"[metal, gear, solid, 4, is, a, game, that, wil...","[story, your, solid, snake, or, old, snake, wh...",star_wars117,5.0,"[Solid-Snake, Otacon, Liquid-Snake, Revolver-O...","[Easter-Egg, Quick-Time-Event, Teleportation, ...","[in, 2014, war, has, become, so, routine, that...","[overview, old, snake, metal, gear, solid, 4, ...","[Kojima-Productions, 8-4,-Ltd., Imagica-Digita...","[Metal-Gear, Metal-Gear-Solid]","[Shooter, Action-Adventure]",Metal-Gear-Solid-4:-Guns-of-the-Patriots,"[PlayStation-3, PlayStation-Network-(PS3)]",[Konami],"[Tom-Clancy's-Splinter-Cell, Tom-Clancy's-Spli...","[Alternate-Historical, Espionage, Modern-

In [55]:
# remove stopwords
stop = stopwords.words('english')

def remove_stop(value):
    if len(value) == 0:
        return value
    else:
        filtered = []
        for word in value:
            if word not in stop:
                filtered.append(word)
        return(filtered)

In [56]:
df['rev_deck'] = df['rev_deck'].apply(lambda x: remove_stop(x))
df['game_deck'] = df['game_deck'].apply(lambda x: remove_stop(x))
df['rev_body'] = df['rev_body'].apply(lambda x: remove_stop(x))
df['game_body'] = df['game_body'].apply(lambda x: remove_stop(x))

df.head()

,date_added,rev_deck,rev_body,reviewer,score,characters,concepts,game_deck,game_body,developers,franchises,genres,name,platforms,publishers,similar_games,themes
0,2008-07-14 21:49:21,"[review, monster]","[hey, bet, gives, tangible, notice, completed,...",snide,5.0,"[Blinky, Pinky, Pac-Man, Inky, Clyde]","[Cutscene, Achievements, Leaderboard, Original...","[one, popular, arcade, games, time, pac, man, ...","[overview, important, icon, arcade, gaming, or...","[Namco, WiZ]","[Pac-Man, ARCADE-GAME-SERIES]","[Strategy, Puzzle]",Pac-Man,"[Game-Boy, Game-Boy-Advance, Game-Gear, Apple-...","[Bandai-Namco-Entertainment, Thunder-Mountain,...","[Lock-'n'-Chase, Lady-Bug, Katamari-Damacy, Sp...",
1,2008-07-20 01:09:02,"[favorite, combat, rpg, time]","[rambling, one, favorite, games, nerd, alert, ...",snide,5.0,,"[Experience-Points, Micromanagement, Dialogue-...","[icewind, dale, ii, last, game, built, infinit...","[summary, icewind, dale, ii, isometric, party,...",[Black-Isle-Studios],"[Icewind-Dale, Dungeons-&-Dragons, Forgotten-R...",[Role-Playing],Icewind-Dale-II,[PC],[Interplay-Entertainment-Corp.],"[Planescape:-Torment, Baldur's-Gate-II:-Shadow...",[Fantasy]
2,2008-07-21 00:34:06,"[simple, fun]","[something, said, single, minded, purpose, gam...",joseppie,4.0,"[Death, Merlin-the-Wizard, Sumner, Garm, Skorn...","[Teleportation, Cloak, Experience-Points, Boss...","[gauntlet, legends, first, 3d, installment, fr...","[story, mage, named, garm, summoned, demon, sk...",[Midway-Games],[Gauntlet],"[Role-Playing, Brawler]",Gauntlet-Legends,"[PlayStation, Dreamcast, Nintendo-64, Arcade]",[Midway-Games],"[Champions-of-Norrath, Ghostbusters:-Sanctum-o...",[Fantasy]
3,2008-07-21 00:36:10,"[best, around, still]","[last, best, mvp, baseball, series, mvp, 2005,...",joseppie,5.0,"[Roger-Clemens, Derrek-Lee, Jonathan-Papelbon,...","[Cover-Athlete, Instant-Replay, Minigames, Art...","[mvp, baseball, 2005, baseball, game, ea, spor...","[new, features, owner, mode, existing, franchi...",[EA-Canada],"[MVP-Baseball, MLB]","[Sports, Baseball]",MVP-Baseball-2005,"[PlayStation-Portable, PlayStation-2, GameCube...",[Electronic-Arts],"[MLB-08:-The-Show, Major-League-Baseball-2K8, ...",
4,2008-07-21 00:44:42,"[thrills, hilarity, ensue]","[need, speed, wanted, acts, injection, freshne...",joseppie,4.0,"[Razor-Callahan, Sergeant-Cross, Mia-Townsend,...","[Speed-Blur, Cutscene, Achievements, Leaderboa...","[cruise, streets, rockport, city, keep, away, ...","[overview, need, speed, wanted, 2005, release,...","[EA-Canada, EA-Black-Box, Ideaworks-Game-Studio]",[Need-for-Speed],"[Driving/Racing, Simulation]",Need-for-Speed:-Most-Wanted,"[Game-Boy-Advance, Mac, PlayStation-2, Xbox-36...","[Electronic-Arts, Pocketeers]","[Forza-Motorsport-2, Test-Drive-Unlimited, Bur...","[Crime, Motorsports]"


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24023 entries, 0 to 26913
Data columns (total 17 columns):
date_added       24023 non-null object
rev_deck         24023 non-null object
rev_body         24023 non-null object
reviewer         24023 non-null object
score            24023 non-null float64
characters       24023 non-null object
concepts         24023 non-null object
game_deck        24023 non-null object
game_body        24023 non-null object
developers       24023 non-null object
franchises       24023 non-null object
genres           24023 non-null object
name             24023 non-null object
platforms        24023 non-null object
publishers       24023 non-null object
similar_games    24023 non-null object
themes           24023 non-null object
dtypes: float64(1), object(16)
memory usage: 3.3+ MB


In [58]:
"""
Removes string digits from list of strings

Input: string x

Returns list of strings without any string digits
"""
def remove_dig(x):
    body = []
    for word in x:
        if word.isdigit() == True:
            continue
        else:
            body.append(word)
    return body

In [59]:
df['rev_deck'] = df['rev_deck'].apply(lambda x: remove_dig(x))
df['game_deck'] = df['game_deck'].apply(lambda x: remove_dig(x))
df['rev_body'] = df['rev_body'].apply(lambda x: remove_dig(x))
df['game_body'] = df['game_body'].apply(lambda x: remove_dig(x))

df.head()

,date_added,rev_deck,rev_body,reviewer,score,characters,concepts,game_deck,game_body,developers,franchises,genres,name,platforms,publishers,similar_games,themes
0,2008-07-14 21:49:21,"[review, monster]","[hey, bet, gives, tangible, notice, completed,...",snide,5.0,"[Blinky, Pinky, Pac-Man, Inky, Clyde]","[Cutscene, Achievements, Leaderboard, Original...","[one, popular, arcade, games, time, pac, man, ...","[overview, important, icon, arcade, gaming, or...","[Namco, WiZ]","[Pac-Man, ARCADE-GAME-SERIES]","[Strategy, Puzzle]",Pac-Man,"[Game-Boy, Game-Boy-Advance, Game-Gear, Apple-...","[Bandai-Namco-Entertainment, Thunder-Mountain,...","[Lock-'n'-Chase, Lady-Bug, Katamari-Damacy, Sp...",
1,2008-07-20 01:09:02,"[favorite, combat, rpg, time]","[rambling, one, favorite, games, nerd, alert, ...",snide,5.0,,"[Experience-Points, Micromanagement, Dialogue-...","[icewind, dale, ii, last, game, built, infinit...","[summary, icewind, dale, ii, isometric, party,...",[Black-Isle-Studios],"[Icewind-Dale, Dungeons-&-Dragons, Forgotten-R...",[Role-Playing],Icewind-Dale-II,[PC],[Interplay-Entertainment-Corp.],"[Planescape:-Torment, Baldur's-Gate-II:-Shadow...",[Fantasy]
2,2008-07-21 00:34:06,"[simple, fun]","[something, said, single, minded, purpose, gam...",joseppie,4.0,"[Death, Merlin-the-Wizard, Sumner, Garm, Skorn...","[Teleportation, Cloak, Experience-Points, Boss...","[gauntlet, legends, first, 3d, installment, fr...","[story, mage, named, garm, summoned, demon, sk...",[Midway-Games],[Gauntlet],"[Role-Playing, Brawler]",Gauntlet-Legends,"[PlayStation, Dreamcast, Nintendo-64, Arcade]",[Midway-Games],"[Champions-of-Norrath, Ghostbusters:-Sanctum-o...",[Fantasy]
3,2008-07-21 00:36:10,"[best, around, still]","[last, best, mvp, baseball, series, mvp, repre...",joseppie,5.0,"[Roger-Clemens, Derrek-Lee, Jonathan-Papelbon,...","[Cover-Athlete, Instant-Replay, Minigames, Art...","[mvp, baseball, baseball, game, ea, sports, ga...","[new, features, owner, mode, existing, franchi...",[EA-Canada],"[MVP-Baseball, MLB]","[Sports, Baseball]",MVP-Baseball-2005,"[PlayStation-Portable, PlayStation-2, GameCube...",[Electronic-Arts],"[MLB-08:-The-Show, Major-League-Baseball-2K8, ...",
4,2008-07-21 00:44:42,"[thrills, hilarity, ensue]","[need, speed, wanted, acts, injection, freshne...",joseppie,4.0,"[Razor-Callahan, Sergeant-Cross, Mia-Townsend,...","[Speed-Blur, Cutscene, Achievements, Leaderboa...","[cruise, streets, rockport, city, keep, away, ...","[overview, need, speed, wanted, release, long,...","[EA-Canada, EA-Black-Box, Ideaworks-Game-Studio]",[Need-for-Speed],"[Driving/Racing, Simulation]",Need-for-Speed:-Most-Wanted,"[Game-Boy-Advance, Mac, PlayStation-2, Xbox-36...","[Electronic-Arts, Pocketeers]","[Forza-Motorsport-2, Test-Drive-Unlimited, Bur...","[Crime, Motorsports]"


In [60]:
from nltk.stem.regexp import RegexpStemmer

In [61]:
stemmer = RegexpStemmer('s$')

In [62]:
"""
Removes s from the end of words (convert plurals to singular)
"""
def stem(x):
    body = []
    for word in x:
        word = stemmer.stem(word)
        body.append(word)
    return(body)

In [63]:
df['rev_deck'] = df['rev_deck'].apply(lambda x: stem(x))
df['game_deck'] = df['game_deck'].apply(lambda x: stem(x))
df['rev_body'] = df['rev_body'].apply(lambda x: stem(x))
df['game_body'] = df['game_body'].apply(lambda x: stem(x))

df.head()

,date_added,rev_deck,rev_body,reviewer,score,characters,concepts,game_deck,game_body,developers,franchises,genres,name,platforms,publishers,similar_games,themes
0,2008-07-14 21:49:21,"[review, monster]","[hey, bet, give, tangible, notice, completed, ...",snide,5.0,"[Blinky, Pinky, Pac-Man, Inky, Clyde]","[Cutscene, Achievements, Leaderboard, Original...","[one, popular, arcade, game, time, pac, man, r...","[overview, important, icon, arcade, gaming, or...","[Namco, WiZ]","[Pac-Man, ARCADE-GAME-SERIES]","[Strategy, Puzzle]",Pac-Man,"[Game-Boy, Game-Boy-Advance, Game-Gear, Apple-...","[Bandai-Namco-Entertainment, Thunder-Mountain,...","[Lock-'n'-Chase, Lady-Bug, Katamari-Damacy, Sp...",
1,2008-07-20 01:09:02,"[favorite, combat, rpg, time]","[rambling, one, favorite, game, nerd, alert, f...",snide,5.0,,"[Experience-Points, Micromanagement, Dialogue-...","[icewind, dale, ii, last, game, built, infinit...","[summary, icewind, dale, ii, isometric, party,...",[Black-Isle-Studios],"[Icewind-Dale, Dungeons-&-Dragons, Forgotten-R...",[Role-Playing],Icewind-Dale-II,[PC],[Interplay-Entertainment-Corp.],"[Planescape:-Torment, Baldur's-Gate-II:-Shadow...",[Fantasy]
2,2008-07-21 00:34:06,"[simple, fun]","[something, said, single, minded, purpose, gam...",joseppie,4.0,"[Death, Merlin-the-Wizard, Sumner, Garm, Skorn...","[Teleportation, Cloak, Experience-Points, Boss...","[gauntlet, legend, first, 3d, installment, fra...","[story, mage, named, garm, summoned, demon, sk...",[Midway-Games],[Gauntlet],"[Role-Playing, Brawler]",Gauntlet-Legends,"[PlayStation, Dreamcast, Nintendo-64, Arcade]",[Midway-Games],"[Champions-of-Norrath, Ghostbusters:-Sanctum-o...",[Fantasy]
3,2008-07-21 00:36:10,"[best, around, still]","[last, best, mvp, baseball, serie, mvp, repres...",joseppie,5.0,"[Roger-Clemens, Derrek-Lee, Jonathan-Papelbon,...","[Cover-Athlete, Instant-Replay, Minigames, Art...","[mvp, baseball, baseball, game, ea, sport, gam...","[new, feature, owner, mode, existing, franchis...",[EA-Canada],"[MVP-Baseball, MLB]","[Sports, Baseball]",MVP-Baseball-2005,"[PlayStation-Portable, PlayStation-2, GameCube...",[Electronic-Arts],"[MLB-08:-The-Show, Major-League-Baseball-2K8, ...",
4,2008-07-21 00:44:42,"[thrill, hilarity, ensue]","[need, speed, wanted, act, injection, freshnes...",joseppie,4.0,"[Razor-Callahan, Sergeant-Cross, Mia-Townsend,...","[Speed-Blur, Cutscene, Achievements, Leaderboa...","[cruise, street, rockport, city, keep, away, c...","[overview, need, speed, wanted, release, long,...","[EA-Canada, EA-Black-Box, Ideaworks-Game-Studio]",[Need-for-Speed],"[Driving/Racing, Simulation]",Need-for-Speed:-Most-Wanted,"[Game-Boy-Advance, Mac, PlayStation-2, Xbox-36...","[Electronic-Arts, Pocketeers]","[Forza-Motorsport-2, Test-Drive-Unlimited, Bur...","[Crime, Motorsports]"


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24023 entries, 0 to 26913
Data columns (total 17 columns):
date_added       24023 non-null object
rev_deck         24023 non-null object
rev_body         24023 non-null object
reviewer         24023 non-null object
score            24023 non-null float64
characters       24023 non-null object
concepts         24023 non-null object
game_deck        24023 non-null object
game_body        24023 non-null object
developers       24023 non-null object
franchises       24023 non-null object
genres           24023 non-null object
name             24023 non-null object
platforms        24023 non-null object
publishers       24023 non-null object
similar_games    24023 non-null object
themes           24023 non-null object
dtypes: float64(1), object(16)
memory usage: 3.3+ MB


In [65]:
df.head()

,date_added,rev_deck,rev_body,reviewer,score,characters,concepts,game_deck,game_body,developers,franchises,genres,name,platforms,publishers,similar_games,themes
0,2008-07-14 21:49:21,"[review, monster]","[hey, bet, give, tangible, notice, completed, ...",snide,5.0,"[Blinky, Pinky, Pac-Man, Inky, Clyde]","[Cutscene, Achievements, Leaderboard, Original...","[one, popular, arcade, game, time, pac, man, r...","[overview, important, icon, arcade, gaming, or...","[Namco, WiZ]","[Pac-Man, ARCADE-GAME-SERIES]","[Strategy, Puzzle]",Pac-Man,"[Game-Boy, Game-Boy-Advance, Game-Gear, Apple-...","[Bandai-Namco-Entertainment, Thunder-Mountain,...","[Lock-'n'-Chase, Lady-Bug, Katamari-Damacy, Sp...",
1,2008-07-20 01:09:02,"[favorite, combat, rpg, time]","[rambling, one, favorite, game, nerd, alert, f...",snide,5.0,,"[Experience-Points, Micromanagement, Dialogue-...","[icewind, dale, ii, last, game, built, infinit...","[summary, icewind, dale, ii, isometric, party,...",[Black-Isle-Studios],"[Icewind-Dale, Dungeons-&-Dragons, Forgotten-R...",[Role-Playing],Icewind-Dale-II,[PC],[Interplay-Entertainment-Corp.],"[Planescape:-Torment, Baldur's-Gate-II:-Shadow...",[Fantasy]
2,2008-07-21 00:34:06,"[simple, fun]","[something, said, single, minded, purpose, gam...",joseppie,4.0,"[Death, Merlin-the-Wizard, Sumner, Garm, Skorn...","[Teleportation, Cloak, Experience-Points, Boss...","[gauntlet, legend, first, 3d, installment, fra...","[story, mage, named, garm, summoned, demon, sk...",[Midway-Games],[Gauntlet],"[Role-Playing, Brawler]",Gauntlet-Legends,"[PlayStation, Dreamcast, Nintendo-64, Arcade]",[Midway-Games],"[Champions-of-Norrath, Ghostbusters:-Sanctum-o...",[Fantasy]
3,2008-07-21 00:36:10,"[best, around, still]","[last, best, mvp, baseball, serie, mvp, repres...",joseppie,5.0,"[Roger-Clemens, Derrek-Lee, Jonathan-Papelbon,...","[Cover-Athlete, Instant-Replay, Minigames, Art...","[mvp, baseball, baseball, game, ea, sport, gam...","[new, feature, owner, mode, existing, franchis...",[EA-Canada],"[MVP-Baseball, MLB]","[Sports, Baseball]",MVP-Baseball-2005,"[PlayStation-Portable, PlayStation-2, GameCube...",[Electronic-Arts],"[MLB-08:-The-Show, Major-League-Baseball-2K8, ...",
4,2008-07-21 00:44:42,"[thrill, hilarity, ensue]","[need, speed, wanted, act, injection, freshnes...",joseppie,4.0,"[Razor-Callahan, Sergeant-Cross, Mia-Townsend,...","[Speed-Blur, Cutscene, Achievements, Leaderboa...","[cruise, street, rockport, city, keep, away, c...","[overview, need, speed, wanted, release, long,...","[EA-Canada, EA-Black-Box, Ideaworks-Game-Studio]",[Need-for-Speed],"[Driving/Racing, Simulation]",Need-for-Speed:-Most-Wanted,"[Game-Boy-Advance, Mac, PlayStation-2, Xbox-36...","[Electronic-Arts, Pocketeers]","[Forza-Motorsport-2, Test-Drive-Unlimited, Bur...","[Crime, Motorsports]"


In [67]:
#df.to_csv('upd_gb_game_reviews.csv', index = False)